In [2]:
# Exam example #1 
# Exercise #2

In [11]:
inputPathPOIs = "exam_ex2_data/POIs.txt"
outputPathPartA = "out_PartA"
outputPathPartB = "out_PartB"

In [12]:
# *****************************************
# Exercise 2 - Part A
# *****************************************

# Read the content of POIs.txt
# POI_ID,latitude,longitude,city,country,category,subcategory
poisRDD = sc.textFile(inputPathPOIs)

In [13]:
# Select only the Italian cities
poisItalyRDD = poisRDD.filter(lambda line: line.split(",")[4]=="Italy").cache()

In [14]:
# Select only the POIs with subcategory="Taxi" or subcategory="Busstop"

def filterSubcategories(line):
    fields = line.split(",")
    subcategory = fields[6]
    
    if (subcategory=="taxi" or subcategory=="busstop"):
        return True
    else:
        return False
    

poisItalySelectedSubcategoriesRDD = poisItalyRDD.filter(filterSubcategories)

In [15]:
# Generate for each of the selected input lines a pair with
# key = city
# value = (taxi=0/1,busstop=0/1)

def extractPairsCitySubcategoriesCounters(line):
    fields = line.split(",")
    city = fields[3]
    subcategory = fields[6]
    
    if (subcategory=="taxi"):
        return (city, (1, 0))
    else:
        return (city, (0, 1))
    
    
    

cityTaxiBusstopRDD = poisItalySelectedSubcategoriesRDD.map(extractPairsCitySubcategoriesCounters)

In [16]:
# For each Italian city, compute
# - number of taxi POIs
# - number of Busstop POIs
cityNumTaxiNumBusstopRDD = cityTaxiBusstopRDD.reduceByKey(lambda v1,v2: (v1[0]+v2[0], v1[1]+v2[1]) )

In [17]:
# Select only the element with
# - number of taxi POIs >=1
# - number of Busstop =0
selectedCityNumTaxiNumBusstopRDD = cityNumTaxiNumBusstopRDD\
.filter(lambda pair: pair[1][0]>=1 and pair[1][1]==0)

In [19]:
# Save the selected cities = the keys of selectedCityNumTaxiNumBusstop
selectedCityNumTaxiNumBusstopRDD.keys()\
.saveAsTextFile(outputPathPartA)

In [24]:
# *****************************************
# Exercise 2 - Part B
# *****************************************

In [25]:
# Count the number of museum POIs for each Italian city

# Generate for each input line a pair with
# key = city
# value = 1 if subcategory="museum"
# value = 0 if subcategory!="museum"

def extractCityMuseum(line):
    fields = line.split(",")
    city = fields[3]
    subcategory = fields[6]
    
    if (subcategory=="museum"):
        return (city, 1)
    else:
        return (city, 0)


cityMuseumPOIsRDD = poisItalyRDD.map(extractCityMuseum)

In [26]:
# Count the number of museum POIs for each Italian city
cityNumMuseumPOIsRDD = cityMuseumPOIsRDD.reduceByKey(lambda v1,v2: v1+v2).cache()

In [27]:
# Each input element represents
# - an italian city
# - the number of museum POIs for that city
# Emit one tuple for each input pair. Each tuple represents
# - numOfCities (set to 1 for each input pair)
# - numMuseumPOIs (for that city) = value of the input pair
numCitiesNumMuseumPOIsRDD = cityNumMuseumPOIsRDD.map(lambda pair: (1, pair[1]))

In [36]:
# Compute total number of Italian cities and total number of "museum" POIs the
# Italy (i.e., in the Italian cities)
# Store it in a local variable of the driver
totalNumCitiesNumMuseumPOIs = numCitiesNumMuseumPOIsRDD\
.reduce(lambda tuple1,tuple2: (tuple1[0]+tuple2[0], tuple1[1]+tuple2[1]) )

In [37]:
# Compute average number of "museum" POIs per city in Italy
# Store it in a local variable of the driver
average = totalNumCitiesNumMuseumPOIs[1]/totalNumCitiesNumMuseumPOIs[0]

In [38]:
# Select only the Italian cities with a number of "museum" POIs
# (subcategory="museum") greater than the average number of "museum" POIs per
# city in Italy
selectedCityNumMuseumPOIsRDD = cityNumMuseumPOIsRDD.\
filter(lambda inputPair: inputPair[1]> average)

In [40]:
# Store the selected cities
selectedCityNumMuseumPOIsRDD.keys()\
.saveAsTextFile(outputPathPartB)